# Death by Mutation [WIP]
> Think you understand Python list comprehensions? Take this one simple quiz to find out

- toc: true
- badges: false
- comments: true
- categories: [python, functional-programming, wip]

*WARNING: This post is a work in progress. Please comeback later*

Let start with a quiz. Try to guess the outputs of the following code block without running it. You can then check whether you're correct by clicking the `Show Output` button.

In [1]:
#collapse-output
fs = [lambda: x for x in [1, 2, 3]]

print(fs[0]())
print(fs[1]())
print(fs[2]())

3
3
3


For those of you who got it right, congratulations! For those of us who didn't, you might want to rerun the code in your own Python REPL to make sure I'm not bluffing. After you're done with that, I recommend taking a moment to think about why it's behaving the way it does before reading on.

Before we start let's formalize what the intended vs actual behavior of the expression `[lambda: x for x in [1, 2, 3]]` is. We start with a list of values `[1, 2, 3]`. For each of these values we want a function that takes no argument and returns that value. The first function returns `1`, the second function returns `2` and so on. At least that's what we would expect. Instead what we got is a list of functions that all return `3`, the last value in the list. Why?

*For those who, at this point, already has a guess for what's really happening, you can jump straight to the answer [here](#The-answer).*

Let's take a little detour and try another quiz. This one's pretty straightforward. Again, guess the outputs.

In [2]:
#collapse-output
x = 1
def f():
    return x

print(f())

x = 2
print(f())

1
2


Since `x` is not defined inside `f`, whenever `f` is called, Python will look up the value of `x` in the environment where `f` is defined, which, in this case, is the global environment. Thus `f()` will always return the current value of the global variable `x`.

And that's exactly what's going on in our list comprehension example. List comprehensions in Python are most equivalent to for loops. `[lambda: x for x in [1, 2, 3]]` can be rewritten as:

In [3]:
fs = []
for x in [1, 2, 3]:
    def f():
        return x
    fs.append(f)

Let's unroll the loop

In [4]:
fs = []

x = 1
def f():
    return x
fs.append(f)

x = 2
def f():
    return x
fs.append(f)

x = 3
def f():
    return x
fs.append(f)

Let's focus on the first function

In [5]:
# fs = []

x = 1
def f():
    return x
# fs.append(f)

x = 2
# def f():
#     return x
# fs.append(f)

x = 3
# def f():
#     return x
# fs.append(f)

Does it look familiar? It's because that's the exact same example in our 2nd quiz!

## The answer

To recap, in our list comprehension example

In [6]:
fs = [lambda: x for x in [1, 2, 3]]

print(fs[0]())
print(fs[1]())
print(fs[2]())

3
3
3


`x` is not in the local scope of `lambda: x` but in the enclosing scope, where `for x in [1, 2, 3]` takes place. Thus `x` always takes the value of `3`, the last value in the list, after the list comprehension finishes execution.

In other words, since `x` is not defined inside `lambda: x`, whenever `fs[0]` is called, Python will look up the value of `x` in the enclosing scope of `lambda: x`, the one where `for x in [1, 2, 3]` takes place. Here the last value `x` was assgined to is `3`. Thus `fs[0]()`, `fs[1]()` and `fs[2]()` will always return `3`.

This can be better ilustrated using a generator instead of list comprehension

In [7]:
fs = (lambda: x for x in [1, 2, 3])

which is equivalent to

In [8]:
def make_gen(xs):
    for x in xs:
        yield lambda: x
fs = make_gen([1, 2, 3])

In [9]:
f1 = next(fs)

print(f"{f1()=}")

f1()=1


In [10]:
f2 = next(fs)

print(f"{f1()=}")
print(f"{f2()=}")

f1()=2
f2()=2


In [11]:
f3 = next(fs)

print(f"{f1()=}")
print(f"{f2()=}")
print(f"{f3()=}")

f1()=3
f2()=3
f3()=3


## A caveat

The for loop is not 100% equivalent to our list comprehension example, however. The difference is that in the latter, `x` isn't in the global scope, i.e. you can't access it outside of the list comprehension

In [12]:
#hide
del x

In [13]:
#hide
[lambda: x for x in [1, 2, 3]]
print(x)

NameError: name 'x' is not defined

A more faithful reproduction would be

In [14]:
def make_funcs():
    fs = []
    for x in [1, 2, 3]:
        def f():
            return x
    fs.append(f)
    return fs
fs = make_funcs()

This, however, doesn't invalidate our explanation earlier.

## The correct way

Now that we know what went wrong, how could we fix the code just a little so that it behaves the way we wanted? *Hint: try converting the example into one that uses `map`.*

The answer is function closure

In [15]:
def f(x):
    def g():
        return x
    return g

fs = [f(x) for x in [1, 2, 3]]

print(fs[0]())
print(fs[1]())
print(fs[2]())

1
2
3


which can be converted into a one-liner using `lambda`

In [16]:
fs = [(lambda x: lambda: x)(x) for x in [1, 2, 3]]

or partial function

In [17]:
from functools import partial

fs = [partial(lambda x: x, x) for x in [1, 2, 3]]